<a href="https://colab.research.google.com/github/EduardaSantosDiniz/vendas_supermecado/blob/main/vendas_supermecado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1️⃣ Definição do Problema
Estamos trabalhando com dados de um supermercado.

**Objetivo do projeto:**



*   Quais produtos terão mais vendas em determinada época do ano?
Quais são os comportamentos de compra dos clientes?
*   Qual o perfil de cliente que mais compra determinado produto?

*   Quais produtos estão com alta demanda e precisam ser reabastecidos com urgência?
*   Como as promoções impactam nas vendas de determinados produtos?

*   Quais produtos estão vendendo abaixo do esperado e precisam ser avaliados ou descontinuados?



**Perguntas a serem respondidas:**


*   Quais são os produtos mais vendidos em cada categoria?

*   Qual é a sazonalidade das vendas? (Produtos que vendem mais em determinadas épocas do ano)
Quais produtos têm maior margem de lucro?
-Como as promoções afetam as vendas?
-Qual o perfil dos clientes (por exemplo, idade, sexo, localização) que mais compram?
-Quais clientes compram produtos de maior valor?
-Qual é o comportamento de compra dos clientes (frequência de compras, tipo de produto)?
-Quais produtos estão em falta com mais frequência?
-Quais produtos têm maior tempo de permanência nas prateleiras sem venda?


In [ ]:
import pandas as pd

df = pd.read_csv('arquivo_mercado.csv')
print(df.head())

    Invoice ID Branch       City Customer type  Gender  \
0  750-67-8428      A     Yangon        Member  Female   
1  226-31-3081      C  Naypyitaw        Normal  Female   
2  631-41-3108      A     Yangon        Normal    Male   
3  123-19-1176      A     Yangon        Member    Male   
4  373-73-7910      A     Yangon        Normal    Male   

             Product line  Unit price  Quantity   Tax 5%     Total       Date  \
0       Health and beauty       74.69         7  26.1415  548.9715   1/5/2019   
1  Electronic accessories       15.28         5   3.8200   80.2200   3/8/2019   
2      Home and lifestyle       46.33         7  16.2155  340.5255   3/3/2019   
3       Health and beauty       58.22         8  23.2880  489.0480  1/27/2019   
4       Sports and travel       86.31         7  30.2085  634.3785   2/8/2019   

    Time      Payment    cogs  gross margin percentage  gross income  Rating  
0  13:08      Ewallet  522.83                 4.761905       26.1415     9.1  
1  10:

In [ ]:
df.columns = ['id', 'ramo1', 'cidade', 'tipo_cliente', 'genero', 'linha_produto', 'preco_unitario', 'quantidade',
              'taxa_5%', 'total', 'data', 'tempo', 'pagamento', 'ramo2', 'ramo3', 'receita_bruta','ramo4']

df = df.drop(columns = ['ramo1','ramo2','ramo3','ramo4'])
print(df.head())

            id     cidade tipo_cliente  genero           linha_produto  \
0  750-67-8428     Yangon       Member  Female       Health and beauty   
1  226-31-3081  Naypyitaw       Normal  Female  Electronic accessories   
2  631-41-3108     Yangon       Normal    Male      Home and lifestyle   
3  123-19-1176     Yangon       Member    Male       Health and beauty   
4  373-73-7910     Yangon       Normal    Male       Sports and travel   

   preco_unitario  quantidade  taxa_5%     total       data  tempo  \
0           74.69           7  26.1415  548.9715   1/5/2019  13:08   
1           15.28           5   3.8200   80.2200   3/8/2019  10:29   
2           46.33           7  16.2155  340.5255   3/3/2019  13:23   
3           58.22           8  23.2880  489.0480  1/27/2019  20:33   
4           86.31           7  30.2085  634.3785   2/8/2019  10:37   

     pagamento  receita_bruta  
0      Ewallet        26.1415  
1         Cash         3.8200  
2  Credit card        16.2155  
3     

# 2️⃣Coleta de Dados
Buscar os dados necessários de fontes diversas (bancos de dados, APIs, arquivos CSV, web scraping, etc.).
Garantir que os dados sejam relevantes para a análise.



# 3️⃣ Limpeza e Preparação dos Dados
Tratar valores ausentes, remover duplicatas e lidar com outliers.
Converter dados para formatos adequados (exemplo: transformar datas, padronizar textos, etc.).
Criar novas variáveis (feature engineering) se necessário.


# 4️⃣ Análise Exploratória de Dados (EDA - Exploratory Data Analysis)
Criar gráficos e estatísticas para entender padrões e tendências.
Identificar correlações entre variáveis.
Descobrir possíveis insights iniciais.



# 5️⃣ Modelagem e Machine Learning (se aplicável)
Escolher modelos adequados (regressão, classificação, clustering, etc.).
Treinar e validar modelos com os dados disponíveis.
Ajustar hiperparâmetros para melhorar o desempenho.


# 6️⃣ Avaliação e Interpretação dos Resultados
Avaliar a performance dos modelos usando métricas adequadas (exemplo: acurácia, erro médio, etc.).
Comparar os resultados com a hipótese inicial.